In [1]:
import pandas as pd 
import cv2 as cv 
import os 
import os.path as osp
import numpy as np
import random 
import matplotlib.pyplot as plt 

In [2]:
tableau20 = [
    (31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),
    (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),
    (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),
    (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),
    (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)
]

In [3]:
seqmap_path = "/truba/home/hbilgi/dev/thesis/mot_data/MOT17/seqmaps/mot17-test-all.txt"
data_path = "/truba/home/hbilgi/dev/thesis/mot_data/MOT17"
results_path = "/truba/home/hbilgi/dev/simple-model/outputs/experiments/mot17_public_test_03-22_10:47:53.493746/test/mot_files"
gt = False

In [4]:
with open(seqmap_path, 'r') as f:
    seqmaps = f.readlines()
seqmaps = [d.replace("\n", "") for d in seqmaps[1:]]

In [5]:
seqmaps

['MOT17-01-DPM',
 'MOT17-03-DPM',
 'MOT17-06-DPM',
 'MOT17-07-DPM',
 'MOT17-08-DPM',
 'MOT17-12-DPM',
 'MOT17-14-DPM',
 'MOT17-01-FRCNN',
 'MOT17-03-FRCNN',
 'MOT17-06-FRCNN',
 'MOT17-07-FRCNN',
 'MOT17-08-FRCNN',
 'MOT17-12-FRCNN',
 'MOT17-14-FRCNN',
 'MOT17-01-SDP',
 'MOT17-03-SDP',
 'MOT17-06-SDP',
 'MOT17-07-SDP',
 'MOT17-08-SDP',
 'MOT17-12-SDP',
 'MOT17-14-SDP']

In [6]:
seqmaps = seqmaps[6:7]
seqmaps

['MOT17-14-DPM']

In [7]:
def save_annotated_image(df, tableau20, save_path):
    font_scale = 1
    font_thickness = 2
    font = cv.FONT_HERSHEY_SIMPLEX

    ped_ids = df['ped_id']
    uniq_ids = np.unique(ped_ids.values)
    new_ids = np.arange(0,uniq_ids.shape[0])
    id_mapping = dict(zip(uniq_ids, new_ids))
    df['ped_id'] = df['ped_id'].map(id_mapping)

    ped_colors = {}
    unique_frames = np.unique(df['frame'].to_numpy())
    for frame in unique_frames:
        frame_df = df.loc[df['frame'] == frame]
        if frame_df.empty:
            continue
        
        img = cv.imread(frame_df.iloc[0]['frame_path'])
        for idx, row in frame_df.iterrows():
            bb_left, bb_top, bb_width, bb_height = row[['bb_left', 'bb_top', 'bb_width', 'bb_height']]
            bb_right = bb_left + bb_width
            bb_bottom = bb_top + bb_height
            text_x = bb_left    
            text_y = bb_bottom - 10  # Adjust this value to control the vertical position of the text
            ped_id = row['ped_id']

            if ped_id in ped_colors.keys():
                color = ped_colors[ped_id]
            else:
                color = random.choice(tableau20)
                ped_colors[ped_id] = color

            img = cv.rectangle(img, (int(bb_left), int(bb_top)), (int(bb_right), int(bb_bottom)), color, 3)
            img = cv.putText(img, f"{ped_id}", (int(text_x), int(text_y)), font, font_scale, color, font_thickness)        
        
        # print(osp.join(save_path,frame_df.iloc[0]['frame_path'].split('/')[-1]))
        cv.imwrite(osp.join(save_path,frame_df.iloc[0]['frame_path'].split('/')[-1]), img)



In [8]:
COL_NAMES = ['frame', 'ped_id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'x', 'y','z']

for seq in seqmaps:

    if gt:
        df = pd.read_csv(osp.join(data_path, 'train', seq, 'gt', 'gt.txt'), header = None, names=COL_NAMES[:-1])
        df['frame_path'] = df['frame'].apply(lambda frame_num: osp.join(data_path, 'train', seq, f'img1/{int(frame_num):06}.jpg'))
        df = df[df['conf']==1]
    else:
        df = pd.read_csv(osp.join(results_path, seq + '.txt'), header = None, names=COL_NAMES)
        df['frame_path'] = df['frame'].apply(lambda frame_num: osp.join(data_path, 'test', seq, f'img1/{int(frame_num):06}.jpg'))
    
    if not osp.exists(osp.join('outputs/vis', seq)):
        os.makedirs(osp.join('outputs/vis', seq))
        
    save_annotated_image(df, tableau20, osp.join('outputs/vis', seq))

In [10]:
path = 'outputs/vis/MOT17-14-DPM'
image_names = sorted(os.listdir(path), key=lambda x: int(x.split('.')[0]))

output_video_path = osp.join('outputs/vis', path.split('/')[-1] + '.mp4')
frame_size = (1920, 1080)  # Width and height of the frames
frame_rate = 25  # Frames per second
codec = cv.VideoWriter_fourcc(*'mp4v')  # Video codec

# Create a VideoWriter object
out = cv.VideoWriter(output_video_path, codec, frame_rate, frame_size)

# Iterate through the list of images and add them to the video
for img_name in image_names:
    img = cv.imread(os.path.join(path, img_name))
    out.write(img)

# Release the VideoWriter object
out.release()

# Optional: Display a message when the video creation is complete
print("Video creation completed.")

Video creation completed.
